In [ ]:
import os
import logging
import time
import glob
from shutil import copyfile
import openslide
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import pandas as pd
import cv2
from skimage.color import rgb2hsv
from skimage.filters import threshold_otsu
from skimage import feature
import copy
import json
import random
from PIL import Image
from skimage.color import rgb2hed
from collections import OrderedDict
from skimage.transform import rescale
from skimage import exposure
import gzip
np.random.seed(0)

In [ ]:
# Functions
def ReadWholeSlideImage(image_path, level=None, RGB=True, read_image=True):
    """
    # =========================
    # Read Whole-Slide Image 
    # =========================
    """
    try:
        wsi_obj = openslide.OpenSlide(image_path)
        n_levels = wsi_obj.level_count
#         print("Number of Levels", n_levels)
#         print("Dimensions:%s, level_dimensions:%s"%(wsi_obj.dimensions, wsi_obj.level_dimensions))
#         print("Level_downsamples:", wsi_obj.level_downsamples)        
#         print("Properties", wsi_obj.properties)     
        if (level is None) or (level > n_levels-1):
            level = n_levels-1
#             print ('Default level selected', level)
        if read_image:
            if RGB:
                image_data = np.transpose(np.array(wsi_obj.read_region((0, 0),
                                   level,
                                   wsi_obj.level_dimensions[level]).convert('RGB')),
                                   axes=[1, 0, 2])
            else: 
                image_data = np.array(wsi_obj.read_region((0, 0),
                           level,
                           wsi_obj.level_dimensions[level]).convert('L')).T
        else:
            image_data = None 
#         print (image_data.shape)
    except openslide.OpenSlideUnsupportedFormatError:
        print('Exception: OpenSlideUnsupportedFormatError')
        return None, None, None

    return wsi_obj, image_data, level

def TissueMaskGeneration(slide_obj, level):
    img_RGB = np.transpose(np.array(slide_obj.read_region((0, 0),
                       level,
                       slide_obj.level_dimensions[level]).convert('RGB')),
                       axes=[1, 0, 2])    
    img_data = img_RGB.copy()    
    morpho_kernel = np.ones((7,7),np.uint8)
    np.place(img_RGB, img_RGB<=0, 255)
    img_RGB = cv2.medianBlur(img_RGB, 15)
    img_HSV = rgb2hsv(img_RGB)
    
    tissue_mask_S = img_HSV[:, :, 1] > threshold_otsu(img_HSV[:, :, 1])
    tissue_mask_S_ccd = cv2.dilate(np.uint8(tissue_mask_S), morpho_kernel, iterations = 5)
    return np.array(tissue_mask_S_ccd), img_data

# def TissueMaskGeneration(slide_obj, level):
#     img_RGB = np.transpose(np.array(slide_obj.read_region((0, 0),
#                        level,
#                        slide_obj.level_dimensions[level]).convert('RGB')),
#                        axes=[1, 0, 2])    
#     img_data = img_RGB.copy()    
#     morpho_kernel = np.ones((7,7),np.uint8)
#     imsave (img_data, out='./step0.png')
#     np.place(img_RGB, img_RGB<=0, 255)
#     imsave (img_RGB, out='./step1.png')
#     img_RGB = cv2.medianBlur(img_RGB, 15)
#     imsave (img_RGB, out='./step2.png')
#     img_HSV = rgb2hsv(img_RGB)
#     imsave (255*img_HSV[:, :, 0], out='./step3h.png')
#     imsave (255*img_HSV[:, :, 1], out='./step3s.png')
#     print (np.max(img_HSV))
#     imshow(img_HSV[:, :, 1])
#     imsave (255*img_HSV[:, :, 2], out='./step3v.png')
    
#     tissue_mask_S = img_HSV[:, :, 1] > threshold_otsu(img_HSV[:, :, 1])
#     imsave (tissue_mask_S, out='./step4.png')
#     tissue_mask_S_ccd = cv2.dilate(np.uint8(tissue_mask_S), morpho_kernel, iterations = 5)
#     imsave (tissue_mask_S_ccd, out='./step5.png')
#     return np.array(tissue_mask_S_ccd), img_data

# Image Helper Functions
def imshow(*args,**kwargs):
    """ Handy function to show multiple plots in on row, possibly with different cmaps and titles
    Usage:
    imshow(img1, title="myPlot")
    imshow(img1,img2, title=['title1','title2'])
    imshow(img1,img2, cmap='hot')
    imshow(img1,img2,cmap=['gray','Blues']) """
    cmap = kwargs.get('cmap', 'gray')
    title= kwargs.get('title','')
    axis_off = kwargs.get('axis_off','')
    if len(args)==0:
        raise ValueError("No images given to imshow")
    elif len(args)==1:
        plt.title(title)
        plt.imshow(args[0], interpolation='none')
    else:
        n=len(args)
        if type(cmap)==str:
            cmap = [cmap]*n
        if type(title)==str:
            title= [title]*n
        plt.figure(figsize=(n*5,10))
        for i in range(n):
            plt.subplot(1,n,i+1)
            plt.title(title[i])
            plt.imshow(args[i], cmap[i])
            if axis_off: 
              plt.axis('off')  
    plt.show()
    
# Image Helper Functions
def imsave(*args, **kwargs):
     """
     Concatenate the images given in args and saves them as a single image in the specified output destination.
     Images should be numpy arrays and have same dimensions along the 0 axis.
     imsave(im1,im2,out="sample.png")
     """
     args_list = list(args)
     for i in range(len(args_list)):
         if type(args_list[i]) != np.ndarray:
             print("Not a numpy array")
             return 0
         if len(args_list[i].shape) == 2:
             args_list[i] = np.dstack([args_list[i]]*3)
             if args_list[i].max() == 1:
                args_list[i] = args_list[i]*255

     out_destination = kwargs.get("out",'')
     try:
         concatenated_arr = np.concatenate(args_list,axis=1)
         im = Image.fromarray(np.uint8(concatenated_arr))
     except Exception as e:
         print(e)
         import ipdb; ipdb.set_trace()
         return 0
     if out_destination:
#          print("Saving to",out_destination)
         im.save(out_destination)
     else:
        return im

In [ ]:
# wsi_path='data/Trainhero/3/3_Wholeslide_Default_Extended.tif'
wsi_label='/media/mirl/Backup Plus/ECDP_Project/data/Trainhero/HEROHE_HER2_STATUS.xlsx'
i = str(10)
wsi_path ='data/Trainhero/' +i+'/'+i+ '_Wholeslide_Default_Extended.tif'
print("here",wsi_path)

In [ ]:
save_path = 'pat_tnn_left/'
# ls = ['1', '9', '2', '19', '3', '33', '4', '43', '5', '57', '6', '60', '7', '62', '8', '63', '10', '70', '11', '71', '12', '82', '13', '83', '14', '94', '15', '96', '16', '105', '17', '121', '18', '122', '20', '123', '21', '124', '22', '125', '23', '126', '24', '127', '25', '129', '26', '131', '27', '132', '28', '133', '29', '135', '30', '136', '31', '138', '32', '139', '34', '140', '35', '141', '36', '142', '37', '143', '38', '144', '39', '145', '40', '146', '41', '151', '42', '152', '44', '153', '45', '155', '46', '158', '47', '159', '48', '160', '49', '161', '50', '162', '51', '164', '52', '165', '53', '166', '54', '167', '55', '168', '56', '169', '58', '171', '59', '172', '61', '173', '64', '174', '65', '175', '66', '176', '67', '177', '68', '182', '69', '183', '72', '184', '73', '185', '74', '186', '75', '187' ]
left_ls=['9', '19', '33', '43', '57', '60', '62', '63', '70', '71', '82', '83', '94', '96', '105', '121', '122', '123', '124', '125', '126', '127', '129', '131', '132', '133', '135', '136', '138', '139', '140', '141', '142', '143', '144', '145', '146', '151', '152', '153', '155', '158', '159', '160', '161', '162', '164', '165', '166', '167', '168', '169', '171', '172', '173', '174', '175', '176', '177', '182', '183', '184', '185', '186', '187']
right_ls = ['1', '2', '3', '4', '5', '6', '7', '8', '10', '11', '12', '13', '14', '15', '16', '17', '18', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '58', '59', '61', '64', '65', '66', '67', '68', '69', '72', '73', '74', '75']
# ls = ['1', '10', '11', '12', '13', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '16', '160', '161', '162', '164', '165', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '175', '176', '177', '178', '18', '181', '182', '183', '184', '185', '186', '187', '188', '189', '19', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '2', '20', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '21', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '22', '220', '221', '222', '223', '224', '226', '228', '23', '230', '231', '232', '233', '234', '235', '236', '24', '240', '241', '242', '243', '244', '245', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '352', '353', '354', '355', '356', '357', '358', '359', '36', '360', '361', '362', '363', '364', '365', '366', '367', '368', '369', '37', '370', '371', '372', '373', '374', '375', '376', '377', '378', '379', '38', '380', '381', '382', '383', '384', '385', '386', '387', '389', '39', '390', '391', '392', '393', '394', '395', '396', '397', '398', '399', '4', '40', '400', '401', '402', '403', '404', '405', '406', '407', '408', '409', '41', '410', '411', '412', '413', '414', '415', '416', '417', '418', '419', '42', '420', '421', '422', '423', '424', '425', '426', '427', '428', '429', '43', '430', '431', '432', '433', '434', '435', '436', '437', '438', '439', '44', '440', '441', '442', '443', '444', '445', '446', '447', '448', '449', '45', '450', '451', '452', '453', '454', '455', '456', '457', '458', '459', '46', '460', '461', '462', '463', '464', '465', '466', '467', '468', '469', '47', '470', '471', '472', '473', '474', '475', '476', '477', '478', '479', '48', '480', '481', '482', '483', '484', '485', '486', '487', '488', '489', '49', '490', '491', '492', '493', '494', '495', '496', '497', '498', '499', '5', '50', '500', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '8', '9']
for i in left_ls:
    wsi_path ='data/Trainhero/' +str(i)+'/'+str(i)+ '_Wholeslide_Default_Extended.tif'
#     wsi_path ='data/Testhero/' +str(i)+'/'+str(i)+ '_Wholeslide_Default_Extended.tif'
    print("here",wsi_path)
    wsi_obj = openslide.OpenSlide(wsi_path)
    print (wsi_obj.level_count, wsi_obj.level_dimensions)
    level=0
    # size= (1024,1024)
    size = wsi_obj.level_dimensions[level]
    patch_size=9192
    x_count = int(size[0]/patch_size)
    y_count = int(size[1]/patch_size)
    print ("size,x_count, y_count",size,x_count, y_count)
    num_count = 0
    for x in range(x_count//4, x_count):
        for y in range(y_count//4, y_count):
            if(num_count==0):
                coord=(x*patch_size, y*patch_size)
                image_data = np.transpose(np.array(wsi_obj.read_region(coord,
                                   level,
                                   (patch_size, patch_size)).convert('RGB')),
                                   axes=[1, 0, 2])
                try:
                #         if np.count_nonzero(image_data)>0:
                    if np.count_nonzero(image_data)>0: 
    #                     print(coord)  
                        morpho_kernel = np.ones((7,7),np.uint8)
                        np.place(image_data, image_data<=0, 255)
                        img_RGB = cv2.medianBlur(image_data, 15)
                        img_HSV = rgb2hsv(image_data)                            
                        tissue_mask_S = img_HSV[:, :, 1] > threshold_otsu(img_HSV[:, :, 1])
                #                #find all your connected components (white blobs in your image)
                        nb_components, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(tissue_mask_S), connectivity=8)
                #                 # Loop through areas in order of size
                #                 sizes = stats[1:, -1]
                #                 sorted_idx = np.argsort(sizes)
                #                 sorted_idx_reversed = sorted_idx[::-1]
                        n_imp = min(25, nb_components)    
                #                 min_size = np.mean(sizes)*0.3 
                #                 tissue_mask_S_cc = np.zeros_like(labels)
                #                 for lidx, cc in zip(sorted_idx_reversed[0:n_imp], [sizes[s] for s in sorted_idx_reversed][0:n_imp]):
                #                     if cc > min_size:
                #                         tissue_mask_S_cc[labels == lidx+1] = 1                            

                        count_fraction = np.count_nonzero(tissue_mask_S)/np.prod(tissue_mask_S.size)
                        print("count_fraction",count_fraction)
                        img_count = 0

                        if count_fraction >= 0.3 :
                            num_count = 1
                            print ("coord,count_fraction",coord,count_fraction)
                            imshow(image_data,img_HSV,tissue_mask_S) 
                            print("nb_components",nb_components)
    #                         z = np.copy(image_data)
                            in_patch_size=299
                            in_x_count = int(9192/in_patch_size)
                            in_y_count = int(9192/in_patch_size)
                            for ix in range(in_x_count):
                                for iy in range(in_y_count):
                                    if img_count<150:
                                        in_coord = (coord[0] + ix*in_patch_size, coord[1]+iy*in_patch_size)
                                        in_image_data = np.transpose(np.array(wsi_obj.read_region(in_coord,
                                           level,
                                           (in_patch_size, in_patch_size)).convert('RGB')),
                                           axes=[1, 0, 2])
                                        try:
                                            if np.count_nonzero(in_image_data)>0: 
                                                morpho_kernel = np.ones((7,7),np.uint8)
                                                np.place(in_image_data, in_image_data<=0, 255)
                                                in_img_RGB = cv2.medianBlur(in_image_data, 15)
                                                in_img_HSV = rgb2hsv(in_image_data)                            
                                                in_tissue_mask_S = in_img_HSV[:, :, 1] > threshold_otsu(in_img_HSV[:, :, 1])
                                                in_nb_components, in_labels, in_stats, in_centroids = cv2.connectedComponentsWithStats(np.uint8(in_tissue_mask_S), connectivity=8)
                                                in_count_fraction = np.count_nonzero(in_tissue_mask_S)/np.prod(in_tissue_mask_S.size)
                                                print("inner count_fraction,in_nb_components",in_count_fraction,in_nb_components)
                                                if in_count_fraction >= 0.3 :
                                                    print ("in_coord,count_fraction",in_coord,in_count_fraction)
                                                    imshow(in_image_data) 
                                                    img_count += 1
                                                    plt.imsave(save_path+str(i)+'_'+str(x)+'_'+str(y)+'_'+str(ix)+'_'+str(iy)+'_'+'.png', in_image_data)
                                                    print("nb_components",in_nb_components)
                                        except ValueError:
                                            print("value error in_coord", in_coord)

            #                 plt.imsave(save_path+i+'_'+str(x)+'_'+str(y)+'_'+'.png', image_data)
            #                 plt.imsave('masks/' + i + '_'+str(x)+'_'+str(y)+'_'+'.png', image_data)
                except ValueError:
                    pass
            #         print ('Error')
            #         imshow (image_data)

In [ ]:
imshow(tissue_mask)

In [ ]:
def get_wsi_cases(mode, patient_range, group_range, patch_size=256, level=6):
    '''
    Get WSI cases with paths in a dictionary
    '''
    wsi_dic = OrderedDict()
    factor = int(patch_size//pow(2, level))
    cfg_path = '../configs/Inference_Config.json'
    with open(cfg_path) as f:
        cfg = json.load(f)
        
    l=patient_range[0];u=patient_range[1]
    tissue_mask_base_path = cfg['cm17_tissue_mask_path'] + '/Level_{}_S/TM_{}'.format(level, mode)
    tissue_RGB_base_path = cfg['cm17_tissue_mask_path'] + '/Level_{}_S/TRGB_{}'.format(level, mode)    
    tissue_mask_image_path = cfg['cm17_tissue_mask_path'] + '/Level_{}_S/TM_png_{}'.format(level, mode)
    
    if not os.path.exists(tissue_mask_base_path):
        os.makedirs(tissue_mask_base_path)
        
    if not os.path.exists(tissue_RGB_base_path):
        os.makedirs(tissue_RGB_base_path)
                
    if not os.path.exists(tissue_mask_image_path):
        os.makedirs(tissue_mask_image_path)

    if mode is 'training':
        # for training
        label_base_path = cfg['cm17_train_annotation_path']
 
    elif mode is 'testing':
        # for testing
        l=patient_range[0]
        u=patient_range[1]

    for i in range(l,u):
        for j in range(group_range[0], group_range[1]):
            wsi_name = 'patient_{:03d}_node_{}'.format(i,j)
            print(wsi_name)            
            mask_path = tissue_mask_base_path+'/patient_{:03d}_node_{}.npy.gz'.format(i,j) 
            tissue_path = tissue_RGB_base_path+'/patient_{:03d}_node_{}.npy.gz'.format(i,j)                        

            path_dic = {}
            if mode is 'training':
                folder = 'center_'+str(int(i//20))
                wsi_path = cfg['cm17_train_data_path']+'/{}/patient_{:03d}_node_{}.tif'.format(folder,i,j)
                label_path = label_base_path + '/patient_{:03d}_node_{}_mask.tif'.format(i,j)
                if not os.path.exists(label_path):
                    label_path = None
            elif mode is 'testing':
                wsi_path = cfg['cm17_test_data_path']+'/patient_{:03d}_node_{}.tif'.format(i,j)
                label_path = None

            slide = openslide.OpenSlide(wsi_path)                           
            # Tissue Mask Generation
            if not (os.path.exists(mask_path)):
                tissue_mask, img_RGB = TissueMaskGeneration(slide, level)           
            else:
                continue
            #   
            strided_mask =  np.ones_like(tissue_mask)
            ones_mask = np.zeros_like(tissue_mask)
            ones_mask[::factor, ::factor] = strided_mask[::factor, ::factor]
            strided_mask = ones_mask*tissue_mask                          
            overlay_strided_mask = tissue_mask*255/2 + strided_mask*255/2                  
            overlay_tissue = img_RGB/2 + np.dstack([tissue_mask]*3)*255/2            
            if label_path is not None:
                label_obj = openslide.OpenSlide(label_path)           
                label_img = np.transpose(np.array(label_obj.read_region((0, 0),
                                   level,
                                   label_obj.level_dimensions[level]).convert('L')))

                overlay_label = label_img*255/2 + tissue_mask*255/2                
                imsave(np.transpose(overlay_tissue, (1,0,2)), overlay_label.T, overlay_strided_mask.T, \
                       out=tissue_mask_image_path + '/' + os.path.basename(wsi_path).split('.')[0]+'.png')
            else:
                imsave(np.transpose(overlay_tissue, (1,0,2)), overlay_strided_mask.T, \
                       out=tissue_mask_image_path + '/' + os.path.basename(wsi_path).split('.')[0]+'.png')
                                            
            if not os.path.exists(mask_path):
                f = gzip.GzipFile(mask_path, "w")
                np.save(file=f, arr=tissue_mask)
                f.close()

            if not os.path.exists(tissue_path):
                f = gzip.GzipFile(tissue_path, "w")
                np.save(file=f, arr=img_RGB)
                f.close()
                                
            # Save_path lists
            path_dic['wsi_path'] = wsi_path
            path_dic['label_path'] = label_path            
            path_dic['tissue_mask_path'] = mask_path
            wsi_dic[wsi_name] = path_dic
    return wsi_dic

In [ ]:
wsi_dic = get_wsi_cases('training', (0,100), (0,5), patch_size=256, level=4)

In [ ]:
wsi_dic = get_wsi_cases('testing', (100,200), (0,5), patch_size=256, level=4)